# CDC Data Processing with Spark Streaming

這個 notebook 展示如何使用 Spark Streaming 處理來自 Kafka 的 CDC (Change Data Capture) 資料。


## 1. 設定 Spark Session


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json

# 建立 Spark Session
spark = SparkSession.builder \
    .appName("CDC-Analysis") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print("✅ Spark Session 已建立")
print(f"Spark UI: http://localhost:8088")


## 2. 讀取 Kafka CDC 資料


In [ ]:
# 讀取客戶資料的變更
customer_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "inventory-server.inventory.customers") \
    .option("startingOffsets", "earliest") \
    .load()

print("✅ Kafka stream 已建立")


## 3. 解析 CDC JSON 資料


In [ ]:
# 解析 JSON 並提取資料
parsed_df = customer_df.select(
    col("timestamp"),
    get_json_object(col("value").cast("string"), "$.op").alias("operation"),
    get_json_object(col("value").cast("string"), "$.after.id").cast("int").alias("customer_id"),
    get_json_object(col("value").cast("string"), "$.after.first_name").alias("first_name"),
    get_json_object(col("value").cast("string"), "$.after.last_name").alias("last_name"),
    get_json_object(col("value").cast("string"), "$.after.email").alias("email"),
    get_json_object(col("value").cast("string"), "$.source.ts_ms").cast("long").alias("source_timestamp")
).filter(col("operation").isNotNull())

print("✅ JSON 解析配置完成")


## 4. 即時資料處理 - 顯示變更


In [ ]:
# 定義處理函數
def show_changes(batch_df, batch_id):
    print(f"\n=== Batch {batch_id} ===")
    
    if batch_df.count() > 0:
        print(f"收到 {batch_df.count()} 筆變更")
        
        # 按操作類型分組
        operations = batch_df.groupBy("operation").count().collect()
        for op in operations:
            print(f"  {op['operation']}: {op['count']} 筆")
        
        # 顯示資料
        batch_df.show(truncate=False)
    else:
        print("沒有新的變更")

# 啟動串流處理
query = parsed_df.writeStream \
    .outputMode("append") \
    .foreachBatch(show_changes) \
    .trigger(processingTime="10 seconds") \
    .start()

print("✅ 串流處理已啟動")
print("💡 現在可以嘗試更新資料庫中的客戶資料來查看即時變更！")


## 5. 測試資料更新

在另一個終端執行以下命令來觸發 CDC 事件：

```bash
# 更新客戶郵箱
docker exec postgres-cdc psql -U postgres -d inventory -c "UPDATE inventory.customers SET email = 'updated@example.com' WHERE id = 1;"

# 新增客戶
docker exec postgres-cdc psql -U postgres -d inventory -c "INSERT INTO inventory.customers (first_name, last_name, email) VALUES ('Test', 'User', 'test@example.com');"

# 刪除客戶
docker exec postgres-cdc psql -U postgres -d inventory -c "DELETE FROM inventory.customers WHERE id = (SELECT MAX(id) FROM inventory.customers);"
```


## 6. 進階分析 - 即時統計


In [ ]:
# 建立即時統計
stats_df = parsed_df \
    .withWatermark("timestamp", "1 minute") \
    .groupBy(
        window(col("timestamp"), "1 minute"),
        col("operation")
    ).count() \
    .orderBy("window")

def show_stats(batch_df, batch_id):
    if batch_df.count() > 0:
        print(f"\n📊 即時統計 (Batch {batch_id}):")
        batch_df.show(truncate=False)

# 停止之前的查詢（如果還在運行）
# query.stop()

# 啟動統計查詢
stats_query = stats_df.writeStream \
    .outputMode("complete") \
    .foreachBatch(show_stats) \
    .trigger(processingTime="30 seconds") \
    .start()

print("✅ 即時統計已啟動")


## 7. 停止串流處理


In [ ]:
# 停止所有查詢
for stream in spark.streams.active:
    print(f"停止串流: {stream.name}")
    stream.stop()

print("✅ 所有串流已停止")


## 8. 批次分析範例

除了即時串流處理，我們也可以對 Kafka 中的歷史資料進行批次分析。


In [ ]:
# 讀取所有歷史資料進行批次分析
batch_df = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "inventory-server.inventory.customers") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load()

# 解析並分析
batch_parsed = batch_df.select(
    get_json_object(col("value").cast("string"), "$.op").alias("operation"),
    get_json_object(col("value").cast("string"), "$.after.id").cast("int").alias("customer_id"),
    get_json_object(col("value").cast("string"), "$.after.email").alias("email")
).filter(col("operation").isNotNull())

print("📊 歷史資料統計:")
batch_parsed.groupBy("operation").count().show()

print(f"總計處理了 {batch_parsed.count()} 筆 CDC 事件")
